In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "prev_step", ENTITY_NAME: "entityA" }, #, ZONE_NAME: "zone" },
        { STEP_NAME: "prev_step", ENTITY_NAME: "entityB" },
        { STEP_NAME: "prev_step", ENTITY_NAME: "entityC" },
        { STEP_NAME: "ext_step", ENTITY_NAME: "entityI", ENV_NAME: "prod2", PIPELINE_NAME: "other_pipeline", ZONE_NAME: "monthly_run" },
        { STEP_NAME: "ext_step2", ENTITY_NAME: "entityJ", PIPELINE_NAME: "other_pipeline2", RUN_ID: "run-00-01-01-000000" }
    ],
    outputs = 
    [
        { ENTITY_NAME: "entityF" },
        { ENTITY_NAME: "entityG" },
        { ENTITY_NAME: "entityH" }
    ],
    custom_inputs =
    [
        { ENTITY_NAME: "entityE", ENTITY_PATH: "/data/something/entityE" },
        { ENTITY_NAME: "entityD", ENTITY_PATH: "/data/something/entityD" }
    ],
    custom_outputs =
    [
        { ENTITY_NAME: "entityK", ENTITY_PATH: "/data/something/entityK" },
        { ENTITY_NAME: "entityL", ENTITY_PATH: "/data/something/entityL" }
    ],
    tmp_inputs =
    [
        { ENTITY_NAME: "entityM" },
        { ENTITY_NAME: "entityN" },
    ],
    tmp_outputs =
    [
        { ENTITY_NAME: "entityF" },
        { ENTITY_NAME: "entityG" },
        { ENTITY_NAME: "entityH" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
#5 read inputs 
prev_step_inputs = substep.inputs(step_name="prev_step")

dfA = spark.read.parquet(prev_step_inputs.entityA).pandas_api()
dfB = spark.read.parquet(prev_step_inputs.entityB).pandas_api()
dfC = spark.read.parquet(prev_step_inputs.entityC).pandas_api()

In [ ]:
#6 make something to create artifacts

dfF = dfA.to_spark()
dfG = dfB.to_spark()
dfH = dfC.to_spark()

In [ ]:
#7 write outputs
outputs = substep.outputs()

dfF.write.parquet(outputs.entityF, mode='overwrite')
dfG.write.parquet(outputs.entityG, mode='overwrite')
dfH.write.parquet(outputs.entityH, mode='overwrite')

custom_outputs = substep.custom_outputs()
custom_inputs = substep.custom_inputs()
tmp_inputs = substep.tmp_inputs()
tmp_outputs = substep.tmp_outputs()

In [ ]:
#8 stop spark
SinaraSpark.stop_session()